In [1]:
using SymPy

In [2]:
m1, m2, k1, k2, k3, c1, c2, c3, t = symbols("m1, m2, k1, k2, k3, c1, c2, c3, t", real=true, positive=true)
f1 = SymFunction("f_1")(t)
f2 = SymFunction("f_2")(t)
x1 = SymFunction("x_1")(t)
x2 = SymFunction("x_2")(t)
;

In [3]:
T = 1//2*m1*diff(x1,t)^2 + 1//2*m2*diff(x2,t)^2

               2                  2
   /d         \       /d         \ 
m1*|--(x_1(t))|    m2*|--(x_2(t))| 
   \dt        /       \dt        / 
---------------- + ----------------
       2                  2        

In [4]:
V = 1//2*k1*(x1-0)^2 + 1//2*k2*(x1-x2)^2 + 1//2*k3*(x2-0)^2

      2                          2         2   
k1*x_1 (t)   k2*(x_1(t) - x_2(t))    k3*x_2 (t)
---------- + --------------------- + ----------
    2                  2                 2     

In [5]:
D = 1//2*c1*diff(x1,t)^2 + 1//2*c2*( diff(x1,t) - diff(x2,t))^2 + 1//2*c3*diff(x2,t)^2

               2                               2                  2
   /d         \       /d            d         \       /d         \ 
c1*|--(x_1(t))|    c2*|--(x_1(t)) - --(x_2(t))|    c3*|--(x_2(t))| 
   \dt        /       \dt           dt        /       \dt        / 
---------------- + ----------------------------- + ----------------
       2                         2                        2        

In [6]:
L = T - V

                                                                   2          
                                                       /d         \       /d  
        2                          2         2      m1*|--(x_1(t))|    m2*|--(
  k1*x_1 (t)   k2*(x_1(t) - x_2(t))    k3*x_2 (t)      \dt        /       \dt 
- ---------- - --------------------- - ---------- + ---------------- + -------
      2                  2                 2               2                  

        2
       \ 
x_2(t))| 
       / 
---------
2        

In [7]:
Eq(diff(L,diff(x1,t), t) - diff(L, x1) + diff(D,diff(x1,t)), f1)

                   /  d              d         \                              
                c2*|2*--(x_1(t)) - 2*--(x_2(t))|                              
   d               \  dt             dt        /               k2*(2*x_1(t) - 
c1*--(x_1(t)) + -------------------------------- + k1*x_1(t) + ---------------
   dt                          2                                          2   
                                                                              

                                   
                 2                 
2*x_2(t))       d                  
--------- + m1*---(x_1(t)) = f_1(t)
                 2                 
               dt                  

In [8]:
diff(L,diff(x1,t), t) - diff(L, x1) + diff(D,diff(x1,t)) - f1

                   /  d              d         \                              
                c2*|2*--(x_1(t)) - 2*--(x_2(t))|                              
   d               \  dt             dt        /               k2*(2*x_1(t) - 
c1*--(x_1(t)) + -------------------------------- + k1*x_1(t) + ---------------
   dt                          2                                          2   
                                                                              

                                   
                 2                 
2*x_2(t))       d                  
--------- + m1*---(x_1(t)) - f_1(t)
                 2                 
               dt                  

In [9]:
diff(L,diff(x2,t), t) - diff(L, x2) + diff(D,diff(x2,t)) - f2

   /    d              d         \                                            
c2*|- 2*--(x_1(t)) + 2*--(x_2(t))|                                            
   \    dt             dt        /      d            k2*(-2*x_1(t) + 2*x_2(t))
---------------------------------- + c3*--(x_2(t)) + -------------------------
                2                       dt                       2            
                                                                              

                                      
                    2                 
                   d                  
 + k3*x_2(t) + m2*---(x_2(t)) - f_2(t)
                    2                 
                  dt                  

List comprehensions

In [10]:
out = [ i for i in 1:3 ]

3-element Vector{Int64}:
 1
 2
 3

In [11]:
out = zeros(3)
for i = 1:3
    out[i] = i
end
out

3-element Vector{Float64}:
 1.0
 2.0
 3.0

Using a list comp to eval Lagrange's equation

In [12]:
eom = [ diff(L, diff(q,t), t) - diff(L,q) + diff(D,diff(q,t)) |> simplify for q in [x1,x2] ] - [f1,f2]

2-element Vector{Sym}:
  c1*Derivative(x_1(t), t) + c2*(Derivative(x_1(t), t) - Derivative(x_2(t), t)) + k1*x_1(t) + k2*(x_1(t) - x_2(t)) + m1*Derivative(x_1(t), (t, 2)) - f_1(t)
 -c2*(Derivative(x_1(t), t) - Derivative(x_2(t), t)) + c3*Derivative(x_2(t), t) - k2*(x_1(t) - x_2(t)) + k3*x_2(t) + m2*Derivative(x_2(t), (t, 2)) - f_2(t)

In [14]:
massMatrix = [ diff(eq, diff(q,t,t)) for eq in eom, q in [x1,x2] ]

2×2 Matrix{Sym}:
 m1   0
  0  m2

In [15]:
stiffnessMatrix = [ diff(eq, q) for eq in eom, q in [x1,x2]]

2×2 Matrix{Sym}:
 k1 + k2      -k2
     -k2  k2 + k3

In [16]:
dampingMatrix = [ diff(eq, diff(q,t)) for eq in eom, q in [x1,x2]]

2×2 Matrix{Sym}:
 c1 + c2      -c2
     -c2  c2 + c3

Going into State-Space
$$ z = [x_1,x_2,\dot{x}_1, \dot{x}_2]^T $$

In [18]:
z1,z2,z3,z4 = symbols("z1,z2,z3,z4", real=true);

In [32]:
sol = solve( eom, [diff(x1,t,t), diff(x2,t,t)], dict=true )[1]

Dict{Any, Any} with 2 entries:
  Derivative(x_1(t), (t, 2)) => -c1*Derivative(x_1(t), t)/m1 - c2*Derivative(x_…
  Derivative(x_2(t), (t, 2)) => c2*Derivative(x_1(t), t)/m2 - c2*Derivative(x_2…

In [33]:
sol[diff(x1,t,t)]

     d               d               d                                        
  c1*--(x_1(t))   c2*--(x_1(t))   c2*--(x_2(t))                               
     dt              dt              dt           k1*x_1(t)   k2*x_1(t)   k2*x
- ------------- - ------------- + ------------- - --------- - --------- + ----
        m1              m1              m1            m1          m1          

              
              
_2(t)   f_1(t)
----- + ------
m1        m1  

In [34]:
rule1 = Dict( diff(x1,t)=>z3, diff(x2,t)=>z4  )

Dict{Sym, Sym} with 2 entries:
  Derivative(x_2(t), t) => z4
  Derivative(x_1(t), t) => z3

In [36]:
rule2 = Dict( x1=>z1, x2=>z2 )

Dict{Sym, Sym} with 2 entries:
  x_2(t) => z2
  x_1(t) => z1

In [41]:
z3d = sol[diff(x1,t,t)] |> subs(rule1) |> subs(rule2) |> simplify

-c1*z3 - c2*z3 + c2*z4 - k1*z1 - k2*z1 + k2*z2 + f_1(t)
-------------------------------------------------------
                           m1                          

In [42]:
z4d = sol[diff(x2,t,t)] |> subs(rule1) |> subs(rule2) |> simplify

c2*z3 - c2*z4 - c3*z4 + k2*z1 - k2*z2 - k3*z2 + f_2(t)
------------------------------------------------------
                          m2                          

In [44]:
F = [z3,z4, z3d, z4d]

4-element Vector{Sym}:
                                                           z3
                                                           z4
 (-c1*z3 - c2*z3 + c2*z4 - k1*z1 - k2*z1 + k2*z2 + f_1(t))/m1
  (c2*z3 - c2*z4 - c3*z4 + k2*z1 - k2*z2 - k3*z2 + f_2(t))/m2

In [46]:
sympy.octave_code(F) |> println

% Not supported in Octave:
% f_1
% f_2
[z3; z4; (-c1.*z3 - c2.*z3 + c2.*z4 - k1.*z1 - k2.*z1 + k2.*z2 + f_1(t))./m1; (c2.*z3 - c2.*z4 - c3.*z4 + k2.*z1 - k2.*z2 - k3.*z2 + f_2(t))./m2]
